# Pivots and Joins

In [ ]:
from datascience import *
from cs104 import *
import numpy as np
%matplotlib inline

## 1. Review groups

<img src="https://media-cldnry.s-nbcnews.com/image/upload/MSNBC/Components/Photo/_new/110118-obama-gift-2p.jpg" width=300>

"President Barack Obama receives a gift from Saudi King Abdullah at the start of their bilateral meeting in Riyadh, Saudi Arabia, on June 3, 2009. The large gold medallion was among several gifts given that day that were valued at $34,500, the State Department later said" --CBS News

In [ ]:
# Read in all gifts, and tidy up table by removing nan values and relabeling columns
all_gifts = Table().read_table('data/obama-gifts.csv')
all_gifts = all_gifts.where('donor_country', are.not_equal_to('nan')) #clean up and remove the nans
all_gifts = all_gifts.select('year_received', 'donor_country', 'value_usd')
all_gifts = all_gifts.relabeled('year_received', 'Year')
all_gifts = all_gifts.relabeled('donor_country', 'Country')
all_gifts = all_gifts.relabeled('value_usd', 'Value')
all_gifts.sample(8)

In [ ]:
# We'll also create a small subset of our Obama Gifts dataset.  
# It contains 3 countries, 3 years, 7 gifts
gifts = all_gifts.where('Year', are.contained_in([2009,2010,2011,2012]))
gifts = gifts.where('Country', are.contained_in(make_array('Denmark', 'Egypt', 'Finland')))
gifts = gifts.sort('Year')
gifts

In [ ]:
gifts.group('Year')

In [ ]:
gifts.group('Country')

In [ ]:
gifts.group('Year', sum)

In [ ]:
all_gifts.group('Year', sum)

In [ ]:
gifts.group('Country', max)

The largest gifts given to Obama:

In [ ]:
all_gifts.group('Country', max).sort('Value max', descending=True)

In [ ]:
gifts.group(make_array('Year', 'Country'))

In [ ]:
gifts.group(make_array('Year', 'Country'), sum)

In [ ]:
all_gifts.group(['Year', 'Country'], max).sort('Value max', descending=True)

## 2. Pivots

Summarize data that has been grouped by two variables in a grid.

In [ ]:
gifts

With only two parameters in `pivot`, the values are counts. 

In [ ]:
gifts.pivot('Country', 'Year')

In [ ]:
gifts.pivot('Year', 'Country')

With four parameters used in `pivot`,
- The first is  the horizontal column labels
- The second is the vertical row labels 
- The third is the column used as values in the grid
- The fourth is the aggregation function used for the values in the grid 

In [ ]:
gifts.pivot('Year', 'Country', 'Value', sum)

In [ ]:
gifts.pivot('Year', 'Country', 'Value', max)

### A pivot from our past: Temperatures in Greenland

We spent some time working with our Upernavik climate data.

In [ ]:
greenland_climate = Table.read_table('data/climate_upernavik.csv')
greenland_climate

In [ ]:
greenland_climate = Table.read_table('data/climate_upernavik.csv').where('Year', are.above(1876))
greenland_climate = greenland_climate.relabeled('Precipitation (millimeters)', "Precip (mm)")
tidy_greenland = greenland_climate.where('Air temperature (C)', are.not_equal_to(999.99))
tidy_greenland = tidy_greenland.where('Sea level pressure (mbar)', are.not_equal_to(9999.9))
tidy_greenland

When plotting line graphs, we needed separate columns for each month.  A pivot gives us a table with that format.

In [ ]:
temps_by_month = tidy_greenland.pivot('Month', 'Year', 'Air temperature (C)', np.mean)
temps_by_month

In [ ]:
two_months = temps_by_month.select('Year', '2', '9')
two_months.show(5)

In [ ]:
two_months.plot('Year')

## 3. Joins

Let's hand pick a few of our favorite gifts.

In [ ]:
best_gifts = gifts.take(0,1,2,3).drop("Year")
best_gifts

If you're curious what they are, here are shortened descriptions for the four we chose.

In [ ]:
best_gifts.with_columns("Gift description", 
                        make_array('Book "Restoring the Military Balance"',
                                   'Yellow alabaster bowl',
                                   'Photograph of Her Majesty and His Royal Highness',
                                   'Hand-blown blue glass bird'))

Here is some other info about countries, specifically the GDP from each country during Obama's first year in office.  ([GDP](https://en.wikipedia.org/wiki/Gross_domestic_product) is gross domestic product, a measure of the value of the final goods and services produced in a country.)

We obtained this data from the [World Bank](https://data.worldbank.org/indicator/NY.GDP.MKTP.CD) and [IMF](https://www.imf.org/en/Publications/WEO/weo-database/2022/April/download-entire-database) sites.

In [ ]:
gdp = Table().with_columns(
    'Country Name', make_array('Denmark', 'Egypt', 'Egypt', 'Greece'),
    'GDP (Billion $)', make_array(321, 189, 198, 331),
    'Source', make_array('World Bank', 'World Bank', 'IMF', 'World Bank'))
gdp

Can we combine `best_gifts` with info about the countries like GDP?

Join will let us merge data from two tables by pairing together rows from each that share a common property.  Here we'll merge a table of gifts with a table of GDP information about the countres giving gifts.

In [ ]:
best_gifts.join('Country', gdp, 'Country Name')

## 4. Open-ended exploration

Suppose we had the following question: *Is there any association between a country's GDP and the total value of gifts they gave to President Obama?*

The World Bank provides [GDP data](https://data.worldbank.org/indicator/NY.GDP.MKTP.CD) we can use.

In [ ]:
gdp = Table().read_table('data/gdp.csv')
gdp.show(3)

**Data wrangling:** Let's subset to the columns we actually need and make these values more readable. 

In [ ]:
gdp_2008 = Table().with_columns(
    'Country Name', gdp.column('Country Name'),
    '2008 GDP Billion USD', gdp.column('2008') / 1e9
)
gdp_2008.show(3)

Let's use **group** to sum up the values of gifts given by each country.  

In [ ]:
all_gifts.show(5)

In [ ]:
gifts_by_country = all_gifts.drop('Year').group('Country', sum)
gifts_by_country.show(5)

Let's **join** these two datasets. 

In [ ]:
joined = gifts_by_country.join('Country', gdp_2008, 'Country Name')
joined

In [ ]:
joined.scatter('2008 GDP Billion USD', 'Value sum')

The scatter plot doesn't show much because most of the data is in a very tiny part of the graph.  It's hard to tell whether the other points are outliers or part of some trend.  In this case, one handy tool is to take the log of each x-value and y-value.  

Recall that for a number $n$, $log(n) = a$ such $10^a = n$.  Here are some values to illustrate how logarithms work:

| n | log(n) |
|---|--------|
| 1 | 0 |
| 10 | 1 |
| 100 | 2 |
| 1,000 | 3 |
| 10,000 | 4 |
| 100,000 | 5 |
| 1,000,000 | 6 |
| 10,000,000 | 7 |
| 100,000,000 | 8 |
| 1,000,000,000 | 9 |

In [ ]:
joined.scatter('2008 GDP Billion USD', 'Value sum', xscale='log', yscale='log')

That makes the points more evenly distributed across both axes of the plots, and it makes any correlation jump out.  We can do a lot with log-log plots like this, but for us, we will be happy to just use it to make the correlation more apparent in this one example.

What about the relationship between oil production and gift values?  You can find oil production for every country or region of the world [here](https://ourworldindata.org/grapher/oil-production-by-country). 

In [ ]:
# Compute the average oil production during the years 2009-2016.
oil = Table().read_table('data/oil_production_by_country.csv')
oil_for_years = oil.where('Year', are.between_or_equal_to(2009, 2016))
oil_by_entity = oil_for_years.drop('Code', 'Year').group('Entity', np.average)
oil_by_entity.sort('Oil production (TWh) average', descending=True)
oil_by_entity.sample(10)

In [ ]:
joined_oil = gifts_by_country.join('Country', oil_by_entity, 'Entity')
joined_oil.scatter('Oil production (TWh) average', 'Value sum')

In [ ]:
plot = joined_oil.scatter('Oil production (TWh) average', 'Value sum',
                   xscale='log', yscale='log')
plot.set_xlim(1/1e3, 1e5)
plot.set_ylim(1e2, 1e6)

If you're curious about the top oil producers in the world...

In [ ]:
joined_oil.sort('Oil production (TWh) average', descending=True)

**Q:** Brainstorm some alternative variables we might want to examine correlations between? 

- Norms of gift giving? Political alliances? Trade deals? 
- Could we track if this relationship changes over time? 
- In this class, we're going to be moving towards some more of these open-ended data questions. 